# Tutorial 3 - Modelos de lenguaje (N-Grama) y Word Embedding


## 1. Modelos de lenguaje

### 1.1 Motivación

En el tutorial 1, vimos que el enfoque **bag-of-words** tiende a considerar las palabras como variables independientes. En otras palabras, este enfoque ingenuo considera que la aparición de una serie de palabras no impacta las probabilidades de cuáles serán la o las palabras siguientes.

<i>Ejemplo</i>: "El gato come sus ..."

La motivación de los **modelos de lenguaje** es dotar la máquina de una mejor comprensión del lenguaje representando las relaciones probabilísticas entre las palabras.

### 1.2 Aplicaciones de los modelos de lenguaje

- En <u>Traducción automática</u>, permite evaluar que tan probables son varias posibilidades de traducción. En el ejemplo, podría indicar que la posibilidad 1 es más probable.

<i>Ejemplo</i>: "El gato del dueño de la casa se come sus croquetas"
1) The house owner's cat eats his kibble 
2) The cat of the owner of the house eats his kible

- En <u>Corrección automática</u>, permite détectar errores probables. En el ejemplo, aunque la palabra "dueña" es correcta, es poco probable que aparezca después de la palabra "del".

<i>Ejemplo</i>: "El gato del dueña de la casa"

- En <u>Finalización automática de textos</u>, permite hacer sugerencias de cómo seguir un texto.

![Finalizacion de textos](T2-languagemodel.png "Logo Title Text 1")

- En <u> Identificación automática del autor</u>, <u> Generación automática de texto</u>, <u> Resumen automático</u> y muchas otras aplicaciones 

### 1.3 Definición

Un modelo de lenguaje es una distribución de probabilidades sobre secuencias de palabras: P($w_1$, ...., $w_n$), donde $w_k$ son las palabras de la secuencia y $w_1$,...,$w_n$ la secuencia completa.


- Existe varias maneras de calcular estas probabilidades. Podemos distinguir los modelos de languaje **n-grama** (aprendidos con métodos estadísticos estandares) y los modelos de lenguaje **neuronales** (aprendidos con redes neuronales).


- Una manera común de calcular P($w_1$, ...., $w_n$) con métodos estadísticos estándares:

P($w_1$) P($w_2$|$w_1$) P($w_3$|$w_1,w_2$) P($w_4$|$w_1,...,w_3$) P($w_n$|$w_1,...,w_{n-1}$) (<i>chain rule</i> en probabilidades)

- En práctica, se utiliza el concepto de **modelo de N-grama** (por ejemplo N=2 o N=3) para referirse al tamaño máximo de las secuencias que se consideran en el modelo de lenguaje.

### 1.4 Aprender un modelo de lenguaje N-Grama

- Modelo bigrama (N=2)

P($w_{n-1}$, $w_n$) = $\frac{C(w_{n-1}, w_n)}{C(w_{n-1})}$

donde C corresponde a contar cuántas veces aparece cierta secuencia en un dataset (o corpus) de entrenamiento.

- Caso general (cualquier valor de N)

P($w_{n-N+1}$,...,$w_n$) = $\frac{C(w_{n-N+1,...,n-1}, w_n)}{C(w_{n-N+1,...,n-1})}$

- Ejemplo:

Corpus: 

1. (s) I am Sam (/s)
2. (s) Sam I am (/s)
3. (s) I do not like eggs (/s)

Modelo de lenguaje bigrama (muestra):

1. P(I | (s)) = 2/3
2. P((/s) | Sam) = 1/2

- Calcular la probabilidad de una frase "El gato come croquetas." con un modelo de lenguaje bigrama

$P(gato | el) P(come | gato) P(croquetas | come) P((/s) | croquetas)$

#### Un ejemplo práctico

In [ ]:
!pip install nltk

In [ ]:
import nltk
nltk.download('reuters')
nltk.download('punkt')

Aprender un modelo de lenguaje trigrama (N=3) en el dataset "Reuters".

In [ ]:
from nltk.corpus import reuters
from nltk import bigrams, trigrams
from collections import Counter, defaultdict

# Create a placeholder for model
model = defaultdict(lambda: defaultdict(lambda: 0))

# Count frequency of co-occurance  
for sentence in reuters.sents():
    for w1, w2, w3 in trigrams(sentence, pad_right=True, pad_left=True):
        model[(w1, w2)][w3] += 1
 
# Let's transform the counts to probabilities
for w1_w2 in model:
    total_count = float(sum(model[w1_w2].values()))
    for w3 in model[w1_w2]:
        model[w1_w2][w3] /= total_count

Realizar algunas predicciones con el modelo de lenguaje:

In [ ]:
dict(model["are","more"])

In [ ]:
dict(model["the","price"])

Podemos iterar sobre el modelo y generar frases aleatorias que parecen coherentes:

In [ ]:
import random

# starting words
text = ["today", "the"]
sentence_finished = False
 
while not sentence_finished:
  # select a random probability threshold  
  r = random.random()
  accumulator = .0

  for word in model[tuple(text[-2:])].keys():
      accumulator += model[tuple(text[-2:])][word]
      # select words that are above the probability threshold
      if accumulator >= r:
          text.append(word)
          break

  if text[-2:] == [None, None]:
      sentence_finished = True
 
print (' '.join([t for t in text if t]))

### 1.5 Limitaciones de los modelos de lenguaje N-Grama


1) Un modelo de lenguaje N-Grama con un valor N mayor es más preciso pero genera problemas de computación.

2) Los modelos N-gramas son representaciones escasa/ingenua del lenguaje. Solo consideran la forma de las palabras y no su significado/semántica


Para mejorar estas limitaciones:

- **Word Embedding** (proyección semántica de las palabras a través de vectores): Word2Vec, GLoVe

- **Modelos de lenguaje neuronales**: BERT, GPT-2, GPT-3



## 2. Word Embedding

### 2.1 Motivación: representar las dimensiones semánticas de cada palabra

1. I want an orange juice.
2. I want an apple ____ .

- Los enfoques <i>bag of words</i> y <i>modelos de lenguaje N-Grama</i> no tienen la capacidad de calcular que las frases 1 y 2 son muy similares porque no tienen una manera de representar que las palabras 'orange' y 'apple' comparten caracterícas (<i>features</i>) comunes.

Los enfoques ingenuos tieden a representar las palabras como vectores "1-Hot". Por ejemplo, supongamos que tenemos un vocabulario de sólo cinco palabras: King, Queen, Man, Woman y Child. Se codificaría la palabra 'Queen' como:

<img src="img/word2vec1.png"/>

- Sería más interesante poder representar la semántica de cada palabra tomando en cuentas ciertas características. 

<img src="img/word2vec2.png"/>


### 2.2 Definición

El concepto de **word embedding** se refiere a un conjunto de técnicas utilizadas para aprender representaciones matemáticas, tipicamente vectores, de cada palabra.

Una de las técnicas más populares es __Word2Vec__ propuesto por un equipo de investigación de Google en 2013 (Efficient Estimation of Word Representations in Vector Space [Mikolov et al., 2013]).

Alternativas populares son __GloVe__ (propuesta por la Universidad de Stanford en 2014) y __FastText__ (propuesta por Facebook en 2016), que extende Word2Vec para considerar de mejor manera las palabras con errores ortográficas.

### 2.3 Algunas propiedades de los word embeddings

- Tener representaciones vectoriales de las palabras permite calcular "razonamiento" de tipo __King - Man + Woman = ?__ y llegar a un resultado cerca de __Queen__.

<img src="img/word2vec4.png"/>

- Tener representaciones vectoriales de las palabras permite realizar razonamientos analógicos de tipo __A es a B, lo que C es a ..__ . Este tipo de propiedades es muy útil para aplicaciones de _Question Answering_ por ejemplo. Las respuestas a las pregutas siguientes <i>¿Cuál es la capital de Chile?</i> o <i>¿Cuáles son los clubs de fútbol en Chile?</i> se pueden responder adicionando vectores.

<img src="img/word2vec6.png"/>

<img src="img/word2vec7.png"/>

<img src="img/word2vec8.png"/>

### 2.4 ¿Cómo se aprenden los vectores? - Redes neuronales

ara construir sus vectores, Word2Vec utiliza un dataset de entrenamiento y algoritmos de aprendizaje basados en redes neuronales (__Continuous Bag of Words__ (CBOW), o modelo __Skip Gram__). El objetivo de esta fase de aprendizaje es aprender cuáles son las palabras _X_ más probables de aparecer en el contexto de una palabra _y_.

<img src="img/word2vec5.png"/>

Por ejemplo, ¿cuál es la probabilidad de tener la palabra 'perro' si aparece la palabra 'pelota' en el contexto?

<code>Los expertos explican que los __perros__ persiguen __pelotas__ en movimiento como parte de un comportamiento instintivo. Aunque no todos los perros tienen tan despiertos su instinto de caza, esto no impide que la mayoría de ellos sí disfruten, y mucho, de los juegos que incluyen persecuciones de una saltarina __pelota__ que bota delante de ellos. </code>

__Algoritmo CBOW__

Las palabras de contexto forman la capa de entrada. Si el tamaño del vocabulario es V, estos serán vectores de dimensión V con sólo uno de los elementos establecido en uno, y el resto todos los ceros. Hay una sola capa oculta y una capa de salida.

<img src="img/word2vec9.png"/>

#### Un ejemplo práctico

La clase <code>word2vec</code> de Gensim permite word embeddings de palabras (ver documentación: https://radimrehurek.com/gensim/models/word2vec.html).

Esta clase tiene varios parametros, en particular:
- <code>sentences</code>: una lista de palabras o de frases que sirve para entrenar el modelo
- <code>sg</code>: define que algoritmos de aprendizaje utilizar (0=CBOW, 1=skip-gram)
- <code>size</code>: define la dimensión de los vectores que se desea extraer
- <code>window</code>: define el número de palabras considerar a la izquierda y a la derecha de una palabra
- <code>min_count</code>: ignorar las palabras que aparecen menos de _min_count_
y otros asociados a la parametrización de la fase de aprendizaje de la red neuronal (que no detallaremos en esta parte del curso):
- <code>alpha</code>: el _learning rate_ utilizado para optimizar los parametros de la red neuronal.
- <code>iter</code>: número de iteraciones (epocas) sobre el dataset para encontrar los parametreos que optimizan la red neuronal.

In [ ]:
from gensim.models import word2vec

Para entrenar nuestro modelo Word2Vec, podemos utilizar nuestros propios datasets o utilizar datasets genericos existentes. Para empezar, utilizaremos 100 MB de textos extraidos de Wikipedia en inglés, para generar vectores de 200 dimensiones.

In [ ]:
sentences = word2vec.Text8Corpus('datasets/text8.txt')

In [ ]:
model = word2vec.Word2Vec(sentences,size=200,hs=1)
#model=word2vec.Word2Vec.load("text8_model")

In [ ]:
print(model)

Ahora que hemos aprendido nuestro modelo, tratemos de resolver la ecuación <code>King - Man + Woman</code>.

En otras palabras buscamos cuál es el vector más similar al vector que adiciona positivamente 'King' y 'Woman' y negativamente 'Man'.

In [ ]:
model.wv.most_similar(positive=['woman','king'],negative=['man'],topn=5)

In [ ]:
model.wv.most_similar(positive=["conflict"])

In [ ]:
model.wv.most_similar(positive=["conflict","weapon"])

In [ ]:
model.wv.most_similar(positive=["conflict"],negative=["weapon"])

In [ ]:
model.wv.most_similar(positive=["life"])

In [ ]:
model.wv.most_similar(positive=["life"],negative=["money"])

Ver los parametros aprendidos por la red neuronal para una palabra dada:

In [ ]:
model.wv['computer']

Guardar el modelo:

In [ ]:
model.save("text8_model")
model=word2vec.Word2Vec.load("text8_model")

In [ ]:
model.wv.doesnt_match("breakfast cereal dinner lunch".split())

In [ ]:
model.wv.doesnt_match("brazil chile france peru argentina".split())

In [ ]:
model.wv.doesnt_match("apple pear banana hammer".split())

In [ ]:
model.wv.similarity('man','woman')

In [ ]:
model.wv.similarity('man','hammer')

In [ ]:
model.wv.similarity('woman','hammer')

In [ ]:
model.wv.similarity('man','engineer')

In [ ]:
model.wv.similarity('woman','engineer')

In [ ]:
model.wv.similarity('man','baby')

In [ ]:
model.wv.similarity('woman','baby')

### 2.5 Limitaciones de los word embeddings

Las técnicas de Word Embeddings dan resultados muy interesantes pero tienen dos principales limitaciones:

1) No permiten tomar en cuenta el orden entre las palabras.

Ejemplo: "Estamos aqui para trabajar y no jugar" vs. "Estamos aqui para jugar y no trabajar"

2) No permiten tomar en cuenta que ciertas palabras cambian de significado según el contexto.

Ejemplo: "I lost my computer __mouse__"

Para mejorar estas limitaciones:

- Combinar Word Embedding con redes neuronales (convolucionales (CNN) o secuenciales (RNN)) que toman en cuenta el orden entre las palabras

- Utilizar modelos de lenguaje neuronales que toman en cuenta el contexto de las palabras: BERT, GPT-2, GPT-3



## 3. Trabajo práctico

1) Aprender modelos de lenguaje N-Grama (N=3, N=4 o N=5) para distintos medios de prensa

- ¿Se puede observar algunas diferencias relevantes en los modelos de lenguaje de cada medio?
- ¿Se podría identificar sesgos ideológicos utilizando estos modelos de lenguaje?

2) Aprender distintos <i>word embeddings</i> utilizando distintos medios de prensa como datasets de entrenamiento

- ¿Se puede observar algunas diferencias relevantes?
- ¿Se podría identificar sesgos ideológicos utilizando estos word embeddings?


In [1]:
from gensim.models import word2vec
import pandas as pd

/home/mvernier/miniconda3/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.10) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [5]:
DATASET="datasets/itv-reinounido.csv" #ej: cnnchile.csv
df = pd.read_csv(DATASET,delimiter="|")
df.head(5)

,date,text,Unnamed: 2
0,2020-09-16,"Longer jail terms for serious offenders, commu...",NaN
1,2020-09-15,Video report by ITV News Political Corresponde...,NaN
2,2020-09-15,Significant further restrictions on our freedo...,NaN
3,2020-09-15,Labour leader Sir Keir Starmer will not take p...,NaN
4,2020-09-15,"A new ""rule of six"" restriction is being imple...",NaN


In [20]:
df_10000=df.head(1000)

In [21]:
# preprocesar la columna "text" para tener un dataset de entrenamiento (lista de tokens)
import spacy

nlp = spacy.load('en_core_web_sm')
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS

In [22]:
train_dataset=""

for index,row in df_10000.iterrows():
    # Text of the news
    text=row[1]
    
    #preprocesamiento spacy
    doc = nlp(text)
    
    for token in doc:
        #print(str(token).lower()+str(token.pos_))
        if (str(token.pos_)!="SPACE" and str(token.pos_)!="PUNCT"):
            train_dataset=train_dataset+str(token).lower()+" "

    #break

In [23]:
import codecs

file = codecs.open("train_dataset.txt", "w", "utf-8")
file.write(train_dataset)
file.close()

In [24]:
with open('train_dataset.txt', 'w') as file: # Use file to refer to the file object
    file.write(train_dataset)

In [25]:
#training word2vec

tokens = word2vec.Text8Corpus('train_dataset.txt')

model = word2vec.Word2Vec(tokens,size=200,hs=1)

In [26]:
print(model)

Word2Vec(vocab=7349, size=200, alpha=0.025)


In [29]:
model.wv.most_similar(positive=['woman'],topn=5)

[('mother', 0.7293562889099121),
 ('man', 0.7246311902999878),
 ('elphicke', 0.6961289644241333),
 ('friend', 0.6951178312301636),
 ('father', 0.6645517945289612)]

In [30]:
model.wv.most_similar(positive=['man'],topn=5)

[('woman', 0.7246313095092773),
 ('mother', 0.6455699801445007),
 ('father', 0.6426215767860413),
 ('wife', 0.6086143255233765),
 ('predecessor', 0.6083647012710571)]

In [31]:
model.save("itv_reinounido_wordembedding")